In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import utils
import os
import tqdm
import struct

In [4]:
def read_aedat(file):
    with open(file, 'rb') as f:
        line = f.readline()
        while line.startswith(b'#'):
            if line == b'#!END-HEADER\r\n':
                break
            else:
                line = f.readline()
        txyp = {
            't': [],
            'x': [],
            'y': [],
            'p': []
        }
        while True:
            header = f.read(28)
            if not header or len(header) == 0:
                break
            # read header
            e_type = struct.unpack('H', header[0:2])[0]
            e_source = struct.unpack('H', header[2:4])[0]
            e_size = struct.unpack('I', header[4:8])[0]
            e_offset = struct.unpack('I', header[8:12])[0]
            e_tsoverflow = struct.unpack('I', header[12:16])[0]
            e_capacity = struct.unpack('I', header[16:20])[0]
            e_number = struct.unpack('I', header[20:24])[0]
            e_valid = struct.unpack('I', header[24:28])[0]

            data_length = e_capacity * e_size
            data = f.read(data_length)
            counter = 0

            if e_type == 1:
                while data[counter:counter + e_size]:
                    aer_data = struct.unpack('I', data[counter:counter + 4])[0]
                    timestamp = struct.unpack('I', data[counter + 4:counter + 8])[0] | e_tsoverflow << 31
                    x = (aer_data >> 17) & 0x00007FFF
                    y = (aer_data >> 2) & 0x00007FFF
                    pol = (aer_data >> 1) & 0x00000001
                    counter = counter + e_size
                    txyp['x'].append(x)
                    txyp['y'].append(y)
                    txyp['t'].append(timestamp)
                    txyp['p'].append(pol)
            else:
                # non-polarity event packet, not implemented
                pass
        txyp['x'] = np.asarray(txyp['x'])
        txyp['y'] = np.asarray(txyp['y'])
        txyp['t'] = np.asarray(txyp['t'])
        txyp['p'] = np.asarray(txyp['p'])
        return txyp

In [5]:
def convert_aedat_dir_to_npy_dir(aedat_data_dir, npy_data_dir):
    # 将aedat_data_dir目录下的.aedat文件读取并转换成np保存的字典，保存在npy_data_dir目录
    print('convert events data from aedat to numpy format.')
    for aedat_file in tqdm.tqdm(utils.list_files(aedat_data_dir, '.aedat')):
        base_name = aedat_file[0: -6]
        events = read_aedat(os.path.join(aedat_data_dir, aedat_file))
        # 读取csv文件，获取各段的label，保存对应的数据和label
        events_csv = np.loadtxt(os.path.join(aedat_data_dir, base_name + '_labels.csv'),
                                dtype=np.uint32, delimiter=',', skiprows=1)
        
        for i in range(events_csv.shape[0]):
            index = 0
            index_l = 0
            index_r = 0
            label = events_csv[i][0]
            t_start = events_csv[i][1]
            t_end = events_csv[i][2]
            
            while True:
                t = events['t'][index]
                if t < t_start:
                    index += 1
                else:
                    index_l = index  # 左闭
                    break
            
            while True:
                t = events['t'][index]
                if t < t_end:
                    index += 1
                else:
                    index_r = index  # 右开
                    break
            
            # [index_l, index_r)
            j = 0
            while True:
                file_name = os.path.join(npy_data_dir, f'{base_name}_{label}_{j}.npy')
                if os.path.exists(file_name):
                    # 防止同一个aedat里存在多个相同label的数据段
                    j += 1
                else:
                    np.save(file=file_name, arr={
                        't': events['t'][index_l:index_r],
                        'x': events['x'][index_l:index_r],
                        'y': events['y'][index_l:index_r],
                        'p': events['p'][index_l:index_r]
                    })
                    break

In [6]:
aedat_data_dir = '../../../DATA/DVS-Gesture/raw/'
npy_data_dir = '../../../DATA/DVS-Gesture/processed/'
convert_aedat_dir_to_npy_dir(aedat_data_dir, npy_data_dir)

  0%|                                                                                          | 0/122 [00:00<?, ?it/s]

convert events data from aedat to numpy format.


100%|██████████████████████████████████████████████████████████████████████████████| 122/122 [2:08:52<00:00, 63.38s/it]


In [9]:
def toTensor(dataPath, duration, bins, mode):
    
    if mode == 'train':
        dataPath = dataPath + 'dvs_train/' #../../../DATA/DVS-Gesture/processed/dvs_train/
    elif mode == 'test':
        dataPath = dataPath + 'dvs_test/' #../../../DATA/DVS-Gesture/processed/dvs_test/
    else:
        print('typo!')
        return 0
    dataFiles = os.listdir(dataPath)
    for datafile in dataFiles:
        data_dir = dataPath + datafile
        dvs_data = np.load(data_dir, allow_pickle=True)
        t, x, y, p = dvs_data.item().values()
        t = t - t.min() #转换为毫秒，并从0开始

        for i in range(bins):#每次取样       
            tensor_data = torch.zeros([bins,2,128,128], dtype = torch.int)

            slice_idx = np.where((t>=i*duration) & (t<(i+1)*duration))[0]

            for j in slice_idx:
                tensor_data[i][p[j]][x[j]][y[j]] = 1

        torch.save(tensor_data, '../../../DATA/DVS-Gesture/processed/tensor_' + mode + '/' + datafile[:-4] + '.pt')
        print(datafile[:-4] + ' successfully saved')

In [10]:
duration = 30*1000
timestep = 1200 * 1000
bins = int(timestep/duration)

In [11]:
dataPath = '../../../DATA/DVS-Gesture/processed/'
toTensor(dataPath = dataPath, duration = duration, bins = bins, mode = 'train')

user01_fluorescent_10_0 successfully saved
user01_fluorescent_11_0 successfully saved
user01_fluorescent_1_0 successfully saved
user01_fluorescent_2_0 successfully saved
user01_fluorescent_3_0 successfully saved
user01_fluorescent_4_0 successfully saved
user01_fluorescent_5_0 successfully saved
user01_fluorescent_6_0 successfully saved
user01_fluorescent_7_0 successfully saved
user01_fluorescent_8_0 successfully saved
user01_fluorescent_8_1 successfully saved
user01_fluorescent_9_0 successfully saved
user01_fluorescent_led_10_0 successfully saved
user01_fluorescent_led_11_0 successfully saved
user01_fluorescent_led_1_0 successfully saved
user01_fluorescent_led_2_0 successfully saved
user01_fluorescent_led_3_0 successfully saved
user01_fluorescent_led_4_0 successfully saved
user01_fluorescent_led_5_0 successfully saved
user01_fluorescent_led_6_0 successfully saved
user01_fluorescent_led_7_0 successfully saved
user01_fluorescent_led_8_0 successfully saved
user01_fluorescent_led_8_1 succe

user04_natural_3_0 successfully saved
user04_natural_4_0 successfully saved
user04_natural_5_0 successfully saved
user04_natural_6_0 successfully saved
user04_natural_7_0 successfully saved
user04_natural_8_0 successfully saved
user04_natural_8_1 successfully saved
user04_natural_9_0 successfully saved
user05_fluorescent_10_0 successfully saved
user05_fluorescent_11_0 successfully saved
user05_fluorescent_1_0 successfully saved
user05_fluorescent_2_0 successfully saved
user05_fluorescent_3_0 successfully saved
user05_fluorescent_4_0 successfully saved
user05_fluorescent_5_0 successfully saved
user05_fluorescent_6_0 successfully saved
user05_fluorescent_7_0 successfully saved
user05_fluorescent_8_0 successfully saved
user05_fluorescent_8_1 successfully saved
user05_fluorescent_9_0 successfully saved
user05_fluorescent_led_10_0 successfully saved
user05_fluorescent_led_11_0 successfully saved
user05_fluorescent_led_1_0 successfully saved
user05_fluorescent_led_2_0 successfully saved
user

user08_lab_7_0 successfully saved
user08_lab_8_0 successfully saved
user08_lab_8_1 successfully saved
user08_lab_9_0 successfully saved
user08_led_10_0 successfully saved
user08_led_11_0 successfully saved
user08_led_1_0 successfully saved
user08_led_2_0 successfully saved
user08_led_3_0 successfully saved
user08_led_4_0 successfully saved
user08_led_5_0 successfully saved
user08_led_6_0 successfully saved
user08_led_7_0 successfully saved
user08_led_8_0 successfully saved
user08_led_8_1 successfully saved
user08_led_9_0 successfully saved
user09_fluorescent_10_0 successfully saved
user09_fluorescent_11_0 successfully saved
user09_fluorescent_1_0 successfully saved
user09_fluorescent_2_0 successfully saved
user09_fluorescent_3_0 successfully saved
user09_fluorescent_4_0 successfully saved
user09_fluorescent_5_0 successfully saved
user09_fluorescent_6_0 successfully saved
user09_fluorescent_7_0 successfully saved
user09_fluorescent_8_0 successfully saved
user09_fluorescent_8_1 successfu

user13_fluorescent_led_8_0 successfully saved
user13_fluorescent_led_8_1 successfully saved
user13_fluorescent_led_9_0 successfully saved
user13_lab_10_0 successfully saved
user13_lab_11_0 successfully saved
user13_lab_1_0 successfully saved
user13_lab_2_0 successfully saved
user13_lab_3_0 successfully saved
user13_lab_4_0 successfully saved
user13_lab_5_0 successfully saved
user13_lab_6_0 successfully saved
user13_lab_7_0 successfully saved
user13_lab_8_0 successfully saved
user13_lab_8_1 successfully saved
user13_lab_9_0 successfully saved
user13_led_10_0 successfully saved
user13_led_11_0 successfully saved
user13_led_1_0 successfully saved
user13_led_2_0 successfully saved
user13_led_3_0 successfully saved
user13_led_4_0 successfully saved
user13_led_5_0 successfully saved
user13_led_6_0 successfully saved
user13_led_7_0 successfully saved
user13_led_8_0 successfully saved
user13_led_8_1 successfully saved
user13_led_9_0 successfully saved
user13_natural_10_0 successfully saved
use

user17_fluorescent_led_4_0 successfully saved
user17_fluorescent_led_5_0 successfully saved
user17_fluorescent_led_6_0 successfully saved
user17_fluorescent_led_7_0 successfully saved
user17_fluorescent_led_8_0 successfully saved
user17_fluorescent_led_8_1 successfully saved
user17_fluorescent_led_9_0 successfully saved
user17_lab_10_0 successfully saved
user17_lab_11_0 successfully saved
user17_lab_1_0 successfully saved
user17_lab_2_0 successfully saved
user17_lab_3_0 successfully saved
user17_lab_4_0 successfully saved
user17_lab_5_0 successfully saved
user17_lab_6_0 successfully saved
user17_lab_7_0 successfully saved
user17_lab_8_0 successfully saved
user17_lab_8_1 successfully saved
user17_lab_9_0 successfully saved
user17_led_10_0 successfully saved
user17_led_11_0 successfully saved
user17_led_1_0 successfully saved
user17_led_2_0 successfully saved
user17_led_3_0 successfully saved
user17_led_4_0 successfully saved
user17_led_5_0 successfully saved
user17_led_6_0 successfully 

user21_fluorescent_led_8_0 successfully saved
user21_fluorescent_led_8_1 successfully saved
user21_fluorescent_led_9_0 successfully saved
user21_lab_10_0 successfully saved
user21_lab_11_0 successfully saved
user21_lab_1_0 successfully saved
user21_lab_2_0 successfully saved
user21_lab_3_0 successfully saved
user21_lab_4_0 successfully saved
user21_lab_5_0 successfully saved
user21_lab_6_0 successfully saved
user21_lab_7_0 successfully saved
user21_lab_8_0 successfully saved
user21_lab_8_1 successfully saved
user21_lab_9_0 successfully saved
user21_natural_10_0 successfully saved
user21_natural_11_0 successfully saved
user21_natural_1_0 successfully saved
user21_natural_2_0 successfully saved
user21_natural_3_0 successfully saved
user21_natural_4_0 successfully saved
user21_natural_5_0 successfully saved
user21_natural_6_0 successfully saved
user21_natural_7_0 successfully saved
user21_natural_8_0 successfully saved
user21_natural_8_1 successfully saved
user21_natural_9_0 successfully 

In [12]:
dataPath = '../../../DATA/DVS-Gesture/processed/'
toTensor(dataPath = dataPath, duration = duration, bins = bins, mode = 'test')

user24_fluorescent_10_0 successfully saved
user24_fluorescent_11_0 successfully saved
user24_fluorescent_1_0 successfully saved
user24_fluorescent_2_0 successfully saved
user24_fluorescent_3_0 successfully saved
user24_fluorescent_4_0 successfully saved
user24_fluorescent_5_0 successfully saved
user24_fluorescent_6_0 successfully saved
user24_fluorescent_7_0 successfully saved
user24_fluorescent_8_0 successfully saved
user24_fluorescent_8_1 successfully saved
user24_fluorescent_9_0 successfully saved
user24_fluorescent_led_10_0 successfully saved
user24_fluorescent_led_11_0 successfully saved
user24_fluorescent_led_1_0 successfully saved
user24_fluorescent_led_2_0 successfully saved
user24_fluorescent_led_3_0 successfully saved
user24_fluorescent_led_4_0 successfully saved
user24_fluorescent_led_5_0 successfully saved
user24_fluorescent_led_6_0 successfully saved
user24_fluorescent_led_7_0 successfully saved
user24_fluorescent_led_8_0 successfully saved
user24_fluorescent_led_8_1 succe

user28_led_2_0 successfully saved
user28_led_3_0 successfully saved
user28_led_4_0 successfully saved
user28_led_5_0 successfully saved
user28_led_6_0 successfully saved
user28_led_7_0 successfully saved
user28_led_8_0 successfully saved
user28_led_8_1 successfully saved
user28_led_9_0 successfully saved
user28_natural_10_0 successfully saved
user28_natural_11_0 successfully saved
user28_natural_1_0 successfully saved
user28_natural_2_0 successfully saved
user28_natural_3_0 successfully saved
user28_natural_4_0 successfully saved
user28_natural_5_0 successfully saved
user28_natural_6_0 successfully saved
user28_natural_7_0 successfully saved
user28_natural_8_0 successfully saved
user28_natural_8_1 successfully saved
user28_natural_9_0 successfully saved
user29_fluorescent_10_0 successfully saved
user29_fluorescent_11_0 successfully saved
user29_fluorescent_1_0 successfully saved
user29_fluorescent_2_0 successfully saved
user29_fluorescent_3_0 successfully saved
user29_fluorescent_4_0 s